In [26]:
import numpy as np
import matplotlib.pyplot as plt

# Intro

This notebook demonstrates usage of the class QuantumSVM defined in
quantum_svm.py. It uses QuantumSVM to build quantum SVMs that classify
points in the iris dataset. QuantumSVM() sets up an SVM whose feature map
is implemented by a parametrized quantum circuit. More precisely, each
datapoint $\vec{x}$ is mapped to a quantum state $U(\vec{x})\left|0\right>$,
and the kernel is given by
$$
K(\vec{x}_i, \vec{x}_j)
=
\left|
\left<0\right|
U(\vec{x}_i)^\dag
U(\vec{x}_j)
\left|0\right>
\right|^2.
$$
The kernel elements are computed by sampling from the probability distribution resulting from acting on $\left|0\right>$ with the circuit $U(\vec{x}_i)^\dag U(\vec{x}_j)$. The circuit is implemented either by simulation or on a quantum computer.

# Prepare iris data
First we prepare the data we will be testing QuantumSVM with. We read the file `bezdekIris.data` into numpy arrays X and Y of datapoints and classifications respectively. We rescale the data to run from 0 to 1 in each dimension, then split the datapoints into training and testing data.

The datapoints are in $\mathbb{R}^4$ and the classes are `'Iris-setosa'`, `'Iris-versicolor'`, and `'Iris-virginica'`.

In [35]:
# Read iris data file
X = []
Y = []
with open('bezdekIris.data') as f:
    for line in f:
        split_line = line.strip().split(',')
        if split_line[-1]:
            X.append([float(string) for string in split_line[:-1]])
            Y.append(split_line[-1])
X = np.array(X)
Y = np.array(Y)

# Rescale to unit interval [0, 1]
X = (X - np.min(X, axis=0))/(np.max(X, axis=0) - np.min(X, axis=0))

# Split into training and testing
inds = list(range(X.shape[0]))
np.random.shuffle(inds)
X_shuffled = X[inds]
Y_shuffled = Y[inds]
num_train = int(0.8 * X.shape[0])
X_train, X_test = X_shuffled[:num_train], X_shuffled[num_train:]
Y_train, Y_test = Y_shuffled[:num_train], Y_shuffled[num_train:]

# Binary classification

A single SVM performs a binary classification of the datapoints. The 
next few cells demonstrate using a quantum SVM to classify the datapoints
into setosa irises and non-setosa irises.

In [36]:
from quantum_svm import QuantumSVM

setosa_svm = QuantumSVM()

QuantumSVM builds a feature map circuit that is repeated layers of layers of $D(\vec{x})\, H^{\otimes n}$ where $D$ is a unitary diagonal in the computational basis that is parametrized by datapoints $\vec{x}\in \mathbb{R}^4$. The promise of a quantum SVM is that we can choose a form for the kernel that is both hard to calculate on a classical computer and potentially useful. Following [\[Havlicek et al 2018\]](https://arxiv.org/abs/1804.11326) we use IQP circuits, which as a class are likely hard to simulate classically.

Options for customizing the feature map circuit can be passed when initiating a QuantumSVM, but the default circuit looks like the following:

In [37]:
print(setosa_svm.ansatz.draw())

     ┌───┐┌──────────────┐                          ┌───┐┌──────────────┐     »
q_0: ┤ H ├┤ Rz(theta[0]) ├──■────────────────────■──┤ H ├┤ Rz(theta[0]) ├─────»
     ├───┤├──────────────┤┌─┴─┐┌──────────────┐┌─┴─┐└───┘└──────────────┘     »
q_1: ┤ H ├┤ Rz(theta[1]) ├┤ X ├┤ Rz(theta[4]) ├┤ X ├──■────────────────────■──»
     ├───┤├──────────────┤└───┘└──────────────┘└───┘┌─┴─┐┌──────────────┐┌─┴─┐»
q_2: ┤ H ├┤ Rz(theta[2]) ├──■────────────────────■──┤ X ├┤ Rz(theta[6]) ├┤ X ├»
     ├───┤├──────────────┤┌─┴─┐┌──────────────┐┌─┴─┐├───┤├──────────────┤└───┘»
q_3: ┤ H ├┤ Rz(theta[3]) ├┤ X ├┤ Rz(theta[5]) ├┤ X ├┤ H ├┤ Rz(theta[3]) ├─────»
     └───┘└──────────────┘└───┘└──────────────┘└───┘└───┘└──────────────┘     »
«                                                                              
«q_0: ───────────────────────■────────────────────■────────────────────────────
«     ┌───┐┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐                          
«q_1: ┤ H ├┤ Rz(theta[1]) ├┤ X ├┤ Rz(the

For now we are just demonstrating binary classification, in particular classification as either `'Iris-setosa'` or not. We make an array of training classifications where 1 represents `'Iris-setosa'` and -1 represents any other type of iris.

In [38]:
Y_train_setosa = np.array([1 if y == 'Iris-setosa' else -1 for y in Y_train])

We now fit the SVM model to training datapoints and classifications using the `train()` method of `QuantumSVM`. This might take a couple minutes.

In [39]:
setosa_svm.train(X_train, Y_train_setosa)

Classifying new datapoints is done using the `predict()` method:

In [40]:
Y_predicted_setosa = setosa_svm.predict(X_test)

Finally, we test the accuracy of our setosa/non-setosa classification:

In [41]:
Y_test_setosa = np.array([1 if y == 'Iris-setosa' else -1 for y in Y_test])
accuracy = np.mean(Y_test_setosa == Y_predicted_setosa)
print(f'Classified setosa versus non-setosa with accuracy {accuracy}')

Classified setosa versus non-setosa with accuracy 1.0


For completeness we now build and test binary classifiers for the other two types of iris, `'Iris-versicolor'` and `'Iris-virginica'`:

In [42]:
versicolor_svm = QuantumSVM()

Y_train_versicolor = np.array([1 if y == 'Iris-versicolor' else -1 for y in Y_train])
versicolor_svm.train(X_train, Y_train_versicolor)

Y_test_versicolor = np.array([1 if y == 'Iris-versicolor' else -1 for y in Y_test])
Y_predicted_versicolor = versicolor_svm.predict(X_test)
accuracy_versicolor = np.mean(Y_test_versicolor == Y_predicted_versicolor)
print(f'Classified versicolor versus non-versicolor with accuracy {accuracy_versicolor}')

Classified versicolor versus non-versicolor with accuracy 0.9666666666666667


In [43]:
virginica_svm = QuantumSVM()

Y_train_virginica = np.array([1 if y == 'Iris-virginica' else -1 for y in Y_train])
virginica_svm.train(X_train, Y_train_virginica)

Y_test_virginica = np.array([1 if y == 'Iris-virginica' else -1 for y in Y_test])
Y_predicted_virginica = virginica_svm.predict(X_test)
accuracy_virginica = np.mean(Y_test_virginica == Y_predicted_virginica)
print(f'Classified virginica versus non-virginica with accuracy {accuracy_virginica}')

Classified virginica versus non-virginica with accuracy 0.9333333333333333


### Changing the feature map

As mentioned above, the feature-map circuit can be customized in several ways. The feature-map depends on a choice of $D$ as well as a choice of how $D$ is
parametrized by datapoints. QuantumSVM has default choices for these two that can be changed using the `diagonal_gates=` and `parameter_map=` options respectively when initializing the QuantumSVM.

The user can also change the number of qubits in the circuit using `num_qubits=`. The default number of qubits is 4.

Finally, we can increase or decrease the number of layers of $D(\vec{x})\,\,H^{\otimes n}$ using the `num_layers=` keyword argument. We demonstrate this in the next cell by building two versicolor classifiers with different numbers of layers.

In [48]:
# Try three layers
versicolor_svm = QuantumSVM(num_layers=3)
print(versicolor_svm.ansatz.draw())
versicolor_svm.train(X_train, Y_train_versicolor)
Y_predicted_versicolor = versicolor_svm.predict(X_test)
accuracy3 = np.mean(Y_test_versicolor == Y_predicted_versicolor)
print(f'Accuracy with 3 layers: {accuracy3}\n')

# Try one layer
versicolor_svm = QuantumSVM(num_layers=1)
print(versicolor_svm.ansatz.draw())
versicolor_svm.train(X_train, Y_train_versicolor)
Y_predicted_versicolor = versicolor_svm.predict(X_test)
accuracy1 = np.mean(Y_test_versicolor == Y_predicted_versicolor)
print(f'Accuracy with 1 layer: {accuracy1}\n')

     ┌───┐┌──────────────┐                          ┌───┐┌──────────────┐     »
q_0: ┤ H ├┤ Rz(theta[0]) ├──■────────────────────■──┤ H ├┤ Rz(theta[0]) ├─────»
     ├───┤├──────────────┤┌─┴─┐┌──────────────┐┌─┴─┐└───┘└──────────────┘     »
q_1: ┤ H ├┤ Rz(theta[1]) ├┤ X ├┤ Rz(theta[4]) ├┤ X ├──■────────────────────■──»
     ├───┤├──────────────┤└───┘└──────────────┘└───┘┌─┴─┐┌──────────────┐┌─┴─┐»
q_2: ┤ H ├┤ Rz(theta[2]) ├──■────────────────────■──┤ X ├┤ Rz(theta[6]) ├┤ X ├»
     ├───┤├──────────────┤┌─┴─┐┌──────────────┐┌─┴─┐├───┤├──────────────┤└───┘»
q_3: ┤ H ├┤ Rz(theta[3]) ├┤ X ├┤ Rz(theta[5]) ├┤ X ├┤ H ├┤ Rz(theta[3]) ├─────»
     └───┘└──────────────┘└───┘└──────────────┘└───┘└───┘└──────────────┘     »
«                                                    ┌───┐┌──────────────┐     »
«q_0: ───────────────────────■────────────────────■──┤ H ├┤ Rz(theta[0]) ├─────»
«     ┌───┐┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐└───┘└──────────────┘     »
«q_1: ┤ H ├┤ Rz(theta[1]) ├┤ X ├┤ Rz(

Note that more layers is not always better.

### Changing the sampler

By default, QuantumSVM uses the sampler primitive `qiskit.primitives.Sampler` to run circuits that estimate kernel entries. This uses a local simulator. To customize the sampler primitive used we can pass a sampler object as an argument `sampler=` when initializing the QuantumSVM. This allows us, for example, to run the circuit on an IBM backend using Qiskit Runtime, if we desire.

Let's demonstrate this by running the circuits on IBM's `'simulator_statevector'` simulator backend over the cloud. This will require an IBM Quantum account. Specifically it will require an IBM Quantum API token. First we set up a Qiskit Runtime Service:

In [ ]:
from qiskit_ibm_runtime import Session, QiskitRuntimeService, Sampler

# replace 'MY_IBM_QUANTUM_TOKEN' with your own IBM Quantum API token
service = QiskitRuntimeService(channel='ibm_quantum')#, token='MY_IBM_QUANTUM_TOKEN')

Now we build and use the SVM in the context of a Qiskit Runtime
session. (This may take some minutes depending on the queue!)

In [14]:
backend = service.get_backend('simulator_statevector')

with Session(service=service, backend=backend) as session:
    
    sampler = Sampler(session=session, options={'execution': {'shots': 1e4}})
    
    setosa_svm = QuantumSVM(sampler=sampler)
    setosa_svm.train(X_train, Y_train_setosa)
    Y_predicted_setosa = setosa_svm.predict(X_test)

    accuracy = np.mean(Y_predicted_setosa == Y_test_setosa)
    print(f'\nClassified setosa versus non-setosa with accuracy {accuracy}')

Job completed                                 
Classified setosa versus non-setosa with accuracy 1.0


We can execute the circuits on a real quantum device by changing the backend from 'simulator_statevector' to, e.g. 'ibmq_manila' or some other IBM device.

# Three-way classification

To classify a datapoint, the SVM assigns it a score whose sign represents which side of the decision boundary the datapoint is on and whose magnitude represents
the datapoint's distance from the decision boundary in feature-space. By default
the `.predict()` method returns 1 or -1 depending on the sign of this score.
We can get the raw score itself by setting `score=True`. To perform a three-way
classification we can build an SVM for each of the three classes and the choose the class based on which SVM gives the highest score.

In [45]:
# Make an SVM for each class
setosa_svm = QuantumSVM()
versicolor_svm = QuantumSVM()
virginica_svm = QuantumSVM()

# Get scores from each SVM
setosa_svm.train(X_train, Y_train_setosa)
Y_predicted_setosa = setosa_svm.predict(X_test, score=True)

Y_train_versicolor = np.array([1 if y == 'Iris-versicolor' else -1 for y in Y_train])
versicolor_svm.train(X_train, Y_train_versicolor)
Y_predicted_versicolor = versicolor_svm.predict(X_test, score=True)

Y_train_virginica = np.array([1 if y == 'Iris-virginica' else -1 for y in Y_train])
virginica_svm.train(X_train, Y_train_virginica)
Y_predicted_virginica = virginica_svm.predict(X_test, score=True)

scores = np.column_stack((
    Y_predicted_setosa,
    Y_predicted_versicolor,
    Y_predicted_virginica
))

# Get name of highest-scoring class for each test datapoint
classes = np.array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
Y_predicted = classes[np.argmax(scores, axis=1)]

accuracy = np.mean(Y_predicted == Y_test)
print(f'The accuracy for three-way classification is {accuracy}')


The accuracy for three-way classification is 0.9666666666666667


# Optimizing circuit depth

For the iris data, the 4 qubits 2 layer default circuit of QuantumSVM is a bit overkill. The iris data is nearly linearly separable. (In fact, the setosa cluster *is* linearly separable from the other two.) This suggests that we might not need a particularly complicated kernel function.

To that end, let's consider the very simple 4-qubit circuit consisting of one layer of Hadamards followed by single-qubit Z rotation gates, one for each dimension in the data. The rotation angles will be $\theta_i = 2\pi x_i$ for each datapoint $(x_0, x_1, x_2, x_3)$. We can use the keyword arguments of QuantumSVM to build an SVM with this as its feature-map circuit:

In [46]:
diagonal_gates = [[0], [1], [2], [3]]

def parameter_map(x):
    return 2*np.pi*x

simple_svm = QuantumSVM(
    diagonal_gates=diagonal_gates,
    parameter_map=parameter_map,
    num_layers=1
)

print(simple_svm.ansatz.draw())

     ┌───┐┌──────────────┐
q_0: ┤ H ├┤ Rz(theta[0]) ├
     ├───┤├──────────────┤
q_1: ┤ H ├┤ Rz(theta[1]) ├
     ├───┤├──────────────┤
q_2: ┤ H ├┤ Rz(theta[2]) ├
     ├───┤├──────────────┤
q_3: ┤ H ├┤ Rz(theta[3]) ├
     └───┘└──────────────┘


Of course, inner products of states prepared by circuits of this form are easily calculated classically, obviating the need for a quantum computer. But we will continue for the purpose of demonstration.

Let's use SVMs with this simple feature map circuit to do three-way classification:

In [47]:
diagonal_gates = [[0], [1], [2], [3]]

def parameter_map(x):
    return 2*np.pi*x

regularizer = 0.25

setosa_svm = QuantumSVM(
    diagonal_gates=diagonal_gates,
    parameter_map=parameter_map,
    num_layers=1
)
setosa_svm.train(X_train, Y_train_setosa, regularizer=regularizer)
Y_predicted_setosa = setosa_svm.predict(X_test, score=True)

versicolor_svm = QuantumSVM(
    diagonal_gates=diagonal_gates,
    parameter_map=parameter_map,
    num_layers=1
)
versicolor_svm.train(X_train, Y_train_versicolor, regularizer=regularizer)
Y_predicted_versicolor = versicolor_svm.predict(X_test, score=True)

virginica_svm = QuantumSVM(
    diagonal_gates=diagonal_gates,
    parameter_map=parameter_map,
    num_layers=1
)
virginica_svm.train(X_train, Y_train_virginica, regularizer=regularizer)
Y_predicted_virginica = virginica_svm.predict(X_test, score=True)

scores = np.column_stack((
    Y_predicted_setosa,
    Y_predicted_versicolor,
    Y_predicted_virginica
))

# Get name of highest-scoring class for each test datapoint
classes = np.array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
Y_predicted = classes[np.argmax(scores, axis=1)]

accuracy = np.mean(Y_predicted == Y_test)
print(f'The accuracy is {accuracy}')

The accuracy is 0.9333333333333333            


We see this does not do much worse than the more complicated circuits we've tried above.